In [6]:
import os
import pandas as pd
import numpy as np
import math
import ast
from util import *
import plotly.express as px
import plotly.graph_objects as go

In [208]:
merged = load_dataframes("merged_sales_ratings")
merged.head()

,Name,Meta_score,User_score,Description,URL,Developer,Type,Rating,Rank,Platform,Year,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Genre
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,singleplayer,E,95,N64,1998,Nintendo,4.10,1.89,1.45,0.16,7.60,"['Action Adventure', 'Fantasy', 'Action']"
1,The Legend of Zelda: Ocarina of Time,94.0,90.0,The Legend of Zelda: Ocarina of Time 3D takes ...,https://www.metacritic.com/game/3ds/the-legend...,GREZZO,singleplayer,E10+,95,N64,1998,Nintendo,4.10,1.89,1.45,0.16,7.60,"['Compilation', 'Open-World', 'Miscellaneous',..."
2,The Legend of Zelda: Ocarina of Time,99.0,91.0,"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,singleplayer,E,269,3DS,2011,Nintendo,2.03,1.27,0.62,0.30,4.21,"['Action Adventure', 'Fantasy', 'Action']"
3,The Legend of Zelda: Ocarina of Time,94.0,90.0,The Legend of Zelda: Ocarina of Time 3D takes ...,https://www.metacritic.com/game/3ds/the-legend...,GREZZO,singleplayer,E10+,269,3DS,2011,Nintendo,2.03,1.27,0.62,0.30,4.21,"['Compilation', 'Open-World', 'Miscellaneous',..."
4,Super Mario Galaxy,97.0,91.0,[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,singleplayer,E,49,Wii,2007,Nintendo,6.16,3.40,1.20,0.76,11.52,"['Platform', 'Action', 'Platformer', '3D']"


In [209]:
#merged["Platform"] = merged["Platform"].apply(str)
#merged["Platform"] = merged["Platform"].apply(ast.literal_eval)
merged["Genre"] = merged["Genre"].apply(str)
merged["Genre"] = merged["Genre"].apply(ast.literal_eval)
merged = merged[merged["Year"] > 1970]
merged = merged[merged["Year"] < 2016]
merged = merged[merged['Meta_score'].notna()]
print(len(merged))

7079


In [210]:
merged_name = merged.sort_values("Year", ascending = True).drop_duplicates(subset=['Name']).copy()
#print(merged_name.head())
merged_name["NA_Sales"] = merged.groupby(["Name"])["NA_Sales"].sum()[merged_name["Name"]].tolist()
merged_name["EU_Sales"] = merged.groupby(["Name"])["EU_Sales"].sum()[merged_name["Name"]].tolist()
merged_name["JP_Sales"] = merged.groupby(["Name"])["JP_Sales"].sum()[merged_name["Name"]].tolist()
merged_name["Other_Sales"] = merged.groupby(["Name"])["Other_Sales"].sum()[merged_name["Name"]].tolist()
merged_name["Global_Sales"] = merged.groupby(["Name"])["Global_Sales"].sum()[merged_name["Name"]].tolist()
#merged_name["NA_Sales"] = merged.groupby(["Name","Year"])["NA_Sales"].sum()[
#    list(zip(merged_name["Name"],merged_name["Year"]))].tolist()
#merged_name["EU_Sales"] = merged.groupby(["Name","Year"])["EU_Sales"].sum()[
#    list(zip(merged_name["Name"],merged_name["Year"]))].tolist()
#merged_name["JP_Sales"] = merged.groupby(["Name","Year"])["JP_Sales"].sum()[
#    list(zip(merged_name["Name"],merged_name["Year"]))].tolist()
#merged_name["Other_Sales"] = merged.groupby(["Name","Year"])["Other_Sales"].sum()[
#    list(zip(merged_name["Name"],merged_name["Year"]))].tolist()
#merged_name["Global_Sales"] = merged.groupby(["Name","Year"])["Global_Sales"].sum()[
#    list(zip(merged_name["Name"],merged_name["Year"]))].tolist()
merged_name
#merged.groupby(["Name"])["NA_Sales"].sum()[merged_name["Name"]]

,Name,Meta_score,User_score,Description,URL,Developer,Type,Rating,Rank,Platform,Year,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Genre
1971,Missile Command,63.00,63.0,[Xbox Live Arcade] In one of the most well-kn...,https://www.metacritic.com/game/xbox-360/missi...,Stainless Games,multiplayer,E,545,2600,1980,Atari,2.81,0.34,0.00,0.06,3.21,"[Shoot-'Em-Up, Static, Shooter, Top-Down, Action]"
3449,Defender,60.60,62.6,Protect the humans on the planet surface by de...,https://www.metacritic.com/game/playstation-2/...,7 Studios,multiplayer,T,1971,2600,1980,Atari,1.34,0.24,0.00,0.06,1.65,"[Misc, Third-Person, Shooter, Sci-Fi, Action]"
5001,Ms. Pac-Man,65.00,72.0,"Guide Ms. Pac-Man through mazes, munching Pac-...",https://www.metacritic.com/game/xbox-360/ms-pa...,Namco Bandai Games,singleplayer,E,1108,2600,1981,Atari,1.54,0.10,0.00,0.02,1.65,"[Puzzle, Action, General, Arcade]"
4014,Frogger,61.00,53.0,"Enjoy the classic game Frogger, revitalized an...",https://www.metacritic.com/game/xbox-360/frogger,Digital Eclipse,multiplayer,E,736,2600,1981,Parker Bros.,5.85,0.39,0.00,0.13,6.36,"[Action, General, Arcade]"
6194,Spider-Man,83.50,85.0,YOU DON'T JUST PLAY IT... YOU LIVE IT \r\n\r\n...,https://www.metacritic.com/game/playstation/sp...,Neversoft Entertainment,singleplayer,E,2234,2600,1981,Parker Bros.,3.00,1.39,0.02,0.18,4.59,"[Action, Action Adventure, Sci-Fi]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3885,Fallout 4,85.50,61.5,"As the sole survivor of Vault 111, you enter a...",https://www.metacritic.com/game/playstation-4/...,Bethesda Game Studios,singleplayer,M,110,PS4,2015,Bethesda Softworks,5.42,5.04,0.25,1.57,12.28,"[Western-Style, Role-Playing, General]"
5950,Samurai Warriors 4-II,73.00,72.0,Naomasa Ii appears as a playable character for...,https://www.metacritic.com/game/playstation-4/...,Omega Force,multiplayer,T,10114,PS4,2015,Tecmo Koei,0.03,0.02,0.22,0.01,0.27,"[Action, 3D, Beat-'Em-Up]"
5987,Senran Kagura: Estival Versus,71.00,72.0,The flame that burns twice as bright burns hal...,https://www.metacritic.com/game/pc/senran-kagu...,Tamsoft,singleplayer,NaN,7126,PSV,2015,Marvelous Interactive,0.13,0.05,0.17,0.05,0.40,"[Action, General]"
6001,Shantae and the Pirate's Curse,80.25,82.0,Embark on a brand new adventure which teams Sh...,https://www.metacritic.com/game/wii-u/shantae-...,WayForward,singleplayer,T,14480,3DS,2015,Intergrow,0.00,0.00,0.03,0.00,0.03,"[Platform, Action, Platformer, 2D]"


In [211]:
min_year = merged_name["Year"].min()
max_year = merged_name["Year"].max()
merged_name["Year_window"] = 1980
window = 5
windowed = merged_name.copy()[0:0]
for x in range(0,max_year-min_year):
    group = merged_name[(merged_name["Year"] >= min_year+x) & (merged_name["Year"] <= min_year+x+window)].copy()
    group["Year_window"] =  min_year+x
    windowed = pd.concat([windowed,group])
    
windowed

,Name,Meta_score,User_score,Description,URL,Developer,Type,Rating,Rank,Platform,Year,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Genre,Year_window
1971,Missile Command,63.00,63.0,[Xbox Live Arcade] In one of the most well-kn...,https://www.metacritic.com/game/xbox-360/missi...,Stainless Games,multiplayer,E,545,2600,1980,Atari,2.81,0.34,0.00,0.06,3.21,"[Shoot-'Em-Up, Static, Shooter, Top-Down, Action]",1980
3449,Defender,60.60,62.6,Protect the humans on the planet surface by de...,https://www.metacritic.com/game/playstation-2/...,7 Studios,multiplayer,T,1971,2600,1980,Atari,1.34,0.24,0.00,0.06,1.65,"[Misc, Third-Person, Shooter, Sci-Fi, Action]",1980
5001,Ms. Pac-Man,65.00,72.0,"Guide Ms. Pac-Man through mazes, munching Pac-...",https://www.metacritic.com/game/xbox-360/ms-pa...,Namco Bandai Games,singleplayer,E,1108,2600,1981,Atari,1.54,0.10,0.00,0.02,1.65,"[Puzzle, Action, General, Arcade]",1980
4014,Frogger,61.00,53.0,"Enjoy the classic game Frogger, revitalized an...",https://www.metacritic.com/game/xbox-360/frogger,Digital Eclipse,multiplayer,E,736,2600,1981,Parker Bros.,5.85,0.39,0.00,0.13,6.36,"[Action, General, Arcade]",1980
6194,Spider-Man,83.50,85.0,YOU DON'T JUST PLAY IT... YOU LIVE IT \r\n\r\n...,https://www.metacritic.com/game/playstation/sp...,Neversoft Entertainment,singleplayer,E,2234,2600,1981,Parker Bros.,3.00,1.39,0.02,0.18,4.59,"[Action, Action Adventure, Sci-Fi]",1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3885,Fallout 4,85.50,61.5,"As the sole survivor of Vault 111, you enter a...",https://www.metacritic.com/game/playstation-4/...,Bethesda Game Studios,singleplayer,M,110,PS4,2015,Bethesda Softworks,5.42,5.04,0.25,1.57,12.28,"[Western-Style, Role-Playing, General]",2014
5950,Samurai Warriors 4-II,73.00,72.0,Naomasa Ii appears as a playable character for...,https://www.metacritic.com/game/playstation-4/...,Omega Force,multiplayer,T,10114,PS4,2015,Tecmo Koei,0.03,0.02,0.22,0.01,0.27,"[Action, 3D, Beat-'Em-Up]",2014
5987,Senran Kagura: Estival Versus,71.00,72.0,The flame that burns twice as bright burns hal...,https://www.metacritic.com/game/pc/senran-kagu...,Tamsoft,singleplayer,NaN,7126,PSV,2015,Marvelous Interactive,0.13,0.05,0.17,0.05,0.40,"[Action, General]",2014
6001,Shantae and the Pirate's Curse,80.25,82.0,Embark on a brand new adventure which teams Sh...,https://www.metacritic.com/game/wii-u/shantae-...,WayForward,singleplayer,T,14480,3DS,2015,Intergrow,0.00,0.00,0.03,0.00,0.03,"[Platform, Action, Platformer, 2D]",2014


In [65]:
print(merged["Year"].min())
print(merged["Year"].max())

1980
2015


In [66]:
#max_size = merged["Name"].apply(len).max()
#merged["Name"] = merged["Name"].apply(lambda x:x.rjust(50, " "))

In [223]:
fig = px.bar(
    windowed.sort_values(by=["Year_window",'Global_Sales'], ascending = [True,False]).groupby("Year_window").head(10).sort_values(by=["Year_window",'Global_Sales'], ascending = [True,True]),
    x="Global_Sales",
    y="Name",
    animation_frame="Year_window", 
    range_x=[0, 80],
    color='Meta_score',
    color_continuous_scale=["red", "yellow", "green"],
    range_color = [0,100],
    #barmode="group",
    orientation="h",
    labels={'Name':'Name',
           "Global_Sales": "Global Sales (in Millions)",
           "Meta_score": "Meta Score",
           "Year_window":"Year"}
)
#for f in fig.frames:
#    f.layout.update(xaxis_range = [0,35])
#fig.update_layout(yaxis_range=[0,10])
#fig.update_layout(xaxis_range=[0,35])
fig.update_layout(margin=dict(l=400))
#fig.update_layout(
#    xaxis = dict(
#        tickmode = 'array',
#        tickvals = [0, 5, 10, 15, 20, 25, 30, 25],
#        ticktext = [0, 5, 10, 15, 20, 25, 30, 25]
#    )
#)
#fig.update_layout(
#    xaxis = dict(
#        range = [0,35],
#        type = 'linear'
#    )
#)
fig.update_yaxes(title_font_color="white")
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.write_html('../plots/sales_with_meta_score.html', auto_play=False)
fig.show()

In [224]:
fig = px.bar(
    windowed.sort_values(by=["Year_window",'Meta_score'], ascending = [True,False]).groupby("Year_window").head(10).sort_values(by=["Year_window",'Meta_score'], ascending = [True,True]),
    x="Global_Sales",
    y="Name",
    animation_frame="Year_window", 
    range_x=[0, 80],
    color='Meta_score',
    color_continuous_scale=["red", "yellow", "green"],
    range_color = [0,100],
    #barmode="group",
    orientation="h",
    labels={'Name':'Name',
           "Global_Sales": "Global Sales (in Millions)",
           "Meta_score": "Meta Score",
           "Year_window":"Year"}
)
fig.update_layout(margin=dict(l=400))
fig.update_yaxes(title_font_color="white")
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.write_html('../plots/meta_score_vs_sales.html', auto_play=False)
fig.show()

In [225]:
fig = px.bar(
    windowed.sort_values(by=["Year_window",'Meta_score'], ascending = [True,False]).groupby("Year_window").head(10).sort_values(by=["Year_window",'Meta_score'], ascending = [True,True]),
    x="Meta_score",
    y="Name",
    animation_frame="Year_window", 
    range_x=[0, 100],
    color='User_score',
    color_continuous_scale=["red", "yellow", "green"],
    range_color = [0,100],
    #barmode="group",
    orientation="h",
    labels={'Name':'Name',
           "User_score": "User Score",
           "Meta_score": "Meta Score",
           "Year_window":"Year"}
)
fig.update_layout(margin=dict(l=400))
fig.update_yaxes(title_font_color="white")
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.write_html('../plots/meta_score_vs_user_score.html', auto_play=False)
fig.show()